In [1]:
import json
import os
from PIL import Image

# YOLOv5에 사용되는 어노테이션 파일에 맞게끔 JSON 파일을 변경하는 함수 (LabelMe -> Darknet)

def convert_labelme_to_yolo(json_path, output_path, class_mapping):
    for file in os.listdir(json_path):
        if file.endswith('.json'):
            with open(os.path.join(json_path, file)) as f:
                data = json.load(f)

            image_id = os.path.splitext(file)[0]
            img = Image.open(os.path.join(json_path, f"{image_id}.jpg"))
            width, height = img.size

            with open(os.path.join(output_path, f"{image_id}.txt"), 'w') as f:
                for shape in data['shapes']:
                    class_name = shape['label']
                    # Label명이 4자리 숫자일 경우, Password로 변경함
                    if class_name.isdigit() and len(class_name) == 4:
                        class_name = "Password"
                    
                    
                    class_id = class_mapping[class_name]
                    
                    

                    if shape['shape_type'] == 'rectangle':
                        x1, y1 = shape['points'][0]
                        x2, y2 = shape['points'][1]

                        x_center = (x1 + x2) / 2
                        y_center = (y1 + y2) / 2
                        bbox_width = abs(x2 - x1)
                        bbox_height = abs(y2 - y1)

                    elif shape['shape_type'] == 'polygon':
                        points = shape['points']
                        x_min = min([point[0] for point in points])
                        x_max = max([point[0] for point in points])
                        y_min = min([point[1] for point in points])
                        y_max = max([point[1] for point in points])

                        x_center = (x_min + x_max) / 2
                        y_center = (y_min + y_max) / 2
                        bbox_width = x_max - x_min
                        bbox_height = y_max - y_min

                    else:
                        continue

                    x_center_norm = x_center / width
                    y_center_norm = y_center / height
                    bbox_width_norm = bbox_width / width
                    bbox_height_norm = bbox_height / height

                    f.write(f"{class_id} {x_center_norm} {y_center_norm} {bbox_width_norm} {bbox_height_norm}\n")


In [5]:
json_path = "./json_input/"
output_path = "./yolo_output/"
class_mapping = {"Password": 0, "Doorlock": 1}

convert_labelme_to_yolo(json_path, output_path, class_mapping)

# Validation data와 Train data를 자동으로 나누는 코드도 구현해야함
# 지금 이건 수동으로 직접 옮겼음